In [1]:
# necessary for DenodoConnection
%pip install psycopg2-binary


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# import packages
import os
import sys
import pandas as pd
import numpy as np
import time

# for sql recipes
from libs.denodo.denodo_connection import DenodoConnection
from string import Template

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")
sys.path

import performance_pipeline.compute_bebid_advancement_decisions.src.processing as to_test

In [4]:
args = ['--s3_output_bebid_advancement_decisions', '']
df_out = to_test.main_test(args, DKU_DST_ap_data_sector='CORN_NA_SUMMER')

/root/dme_sagemaker/dme_sagemaker/performance_pipeline/compute_bebid_advancement_decisions/src/processing.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stack_piv[curr_stage_col][(adj_mask) & (future_mask)] = 13
/root/dme_sagemaker/dme_sagemaker/performance_pipeline/compute_bebid_advancement_decisions/src/processing.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (adj_mask) & (future_mask == False)]


In [6]:
template = '''WITH "rv_trait_config_backpop" as (
        SELECT ${DKU_DST_analysis_year} as "analysis_year", "ap_data_sector_id","ap_data_sector_name","trait","trait_guid","description",
                    "ap_analysis","distribution_type","viz_column","direction","conv_operator","conv_factor","spirit_uom","viz_uom","yield_trait",
                    "level","aae_use","dme_metric","dme_chkfl","dme_reg_x","dme_reg_y","dme_rm_est","dme_weighted_trait","is_gse_analysis_require","gse_trait_type"
                FROM  "managed"."rv_ap_sector_trait_config"
                WHERE "analysis_year"=MAX(2019, ${DKU_DST_analysis_year})
        
    )
    
    SELECT DISTINCT 
        "rv_ap_all_pvs"."ap_data_sector" AS "ap_data_sector",
        "rv_ap_all_pvs"."analysis_type" AS "analysis_type",
        "rv_ap_sector_trait_config"."analysis_year" AS "analysis_year",
        "rv_ap_all_pvs"."source_id" AS "source_id",
        LOWER("rv_ap_all_pvs"."market_seg") AS "market_seg",
        "rv_ap_all_pvs"."pipeline_runid" AS "pipeline_runid",
        "rv_ap_all_pvs"."trait" AS "trait",
        "rv_ap_all_pvs"."model" AS "model",
        CASE 
            WHEN "rv_ap_all_pvs"."source_year"=2022 THEN "rv_ap_all_pvs"."entry_identifier"
            ELSE COALESCE("rv_corn_material_tester"."be_bid", "rv_ap_all_pvs"."entry_identifier")
        END as "entry_identifier",
        
        --"rv_ap_all_pvs"."entry_identifier" AS "entry_identifier",
        CASE 
            WHEN "rv_ap_all_pvs"."material_type" LIKE '%ale'
            THEN LOWER("rv_ap_all_pvs"."material_type")
            ELSE 'entry'
            END AS "material_type",
        "rv_ap_all_pvs"."count" AS "count",
        "rv_ap_all_pvs"."loc" AS "loc",
        "rv_ap_all_pvs"."prediction" AS "prediction",
        "rv_ap_all_pvs"."stderr" AS "stderr",
        "rv_ap_sector_trait_config"."metric" AS "metric",
        "rv_ap_sector_trait_config"."dme_chkfl" AS "dme_chkfl",
        "rv_ap_sector_trait_config"."dme_reg_x" AS "dme_reg_x",
        "rv_ap_sector_trait_config"."dme_reg_y" AS "dme_reg_y",
        "rv_ap_sector_trait_config"."dme_rm_est" AS "dme_rm_est"
      FROM (
        SELECT "rv_ap_all_pvs".*
          FROM "advancement"."rv_ap_all_pvs" "rv_ap_all_pvs"
          WHERE (((LOWER("analysis_type") = 'multiyear') AND (LOWER("loc") != 'all')) 
                 OR ((LOWER("loc") = 'all') AND (LOWER("analysis_type") != 'multiyear')))
              AND "ap_data_sector" = '${DKU_DST_ap_data_sector}'
              AND CAST("source_year" AS integer) =  ${DKU_DST_analysis_year} 
              AND "analysis_type" = '${DKU_DST_analysis_type}'
        ) "rv_ap_all_pvs"
      INNER JOIN (
        SELECT DISTINCT
            CAST("analysis_year" AS integer) AS "analysis_year",
            "ap_data_sector_id" AS "ap_data_sector_id",
            "ap_data_sector_name" AS "ap_data_sector_name",
            "trait" AS "trait",
            "yield_trait" AS "yield_trait",
            "level" AS "level",
            "dme_metric" AS "dme_metric",
            "dme_chkfl" AS "dme_chkfl",
            "dme_reg_x" AS "dme_reg_x",
            "dme_reg_y" AS "dme_reg_y",
            NULL AS "dme_qualifier",
            "dme_rm_est" AS "dme_rm_est",
            "dme_weighted_trait" AS "dme_weighted_trait",
            CASE 
                WHEN "dme_metric" = 'performance'
                  THEN 'performance'
                WHEN "dme_metric" IN ('agronomic', 'disease', 'risk') 
                  THEN 'risk'
                WHEN "dme_metric" = 'qualifier'
                  THEN 'qualifier'
                ELSE 'other'
          END AS "metric"
          FROM "rv_trait_config_backpop"
          WHERE "ap_data_sector_name" = '${DKU_DST_ap_data_sector}'
              AND CAST("analysis_year" AS integer) =  ${DKU_DST_analysis_year}
              AND ("dme_metric" != 'na' OR "dme_reg_x" = true OR "dme_reg_y" = true OR "dme_rm_est" != 0 OR "dme_weighted_trait" != 0)
        ) "rv_ap_sector_trait_config"
        ON (LOWER("rv_ap_all_pvs"."ap_data_sector") = LOWER("rv_ap_sector_trait_config"."ap_data_sector_name"))
          AND (LOWER("rv_ap_all_pvs"."trait") = LOWER("rv_ap_sector_trait_config"."trait"))
          AND (CAST("rv_ap_all_pvs"."source_year" AS integer) = "rv_ap_sector_trait_config"."analysis_year")
      LEFT JOIN (
          SELECT DISTINCT
              "abbr_code" as "abbr_code",
              "be_bid" as "be_bid"
          FROM "advancement"."rv_corn_material_tester"
      
      ) "rv_corn_material_tester"
          ON "rv_corn_material_tester"."abbr_code"="rv_ap_all_pvs"."entry_identifier"
  '''

In [10]:
# sql recipe...
mapping = {
                'DKU_DST_ap_data_sector': 'CORN_NA_SUMMER',
                'DKU_DST_analysis_year': '2022',
                'DKU_DST_analysis_type': 'GenoPred',
                'source_ids': '',
                'breakout_level': ''
            }

t = Template(template)
sql = t.substitute(**mapping)

start = time.time()
with DenodoConnection() as dc:
    df = dc.get_data(sql)

end = time.time()
print("Elapsed time:", end - start)